## 생물정보학 및 실습 I final project (sojeong Lee)

#### LIN28 CLIP-seq data를 활용하여 논문 figure와 유사한 형태로 재현하기

* Misson3을 참고하여, 다른 cell의 CLIP-seq data set에서 Fig S2A와 유사한 형태로 재현하는 것을 목표로 함. 
* 원본 데이터에 맞춰서 데이터를 처리하고 시각화해보기

2024-05-30
Human K562 cell Lin28B CLIP-seq data from ENCODE


1. change working directory

In [2]:
%cd /qbio/lsj975/study/down_data/

/qbio/lsj975/study/down_data


In [3]:
!ls -al

total 994852
drwxr-xr-x 1 lsj975 qbio       146 May 30 19:38 .
drwxr-xr-x 1 lsj975 qbio       288 May 30 19:43 ..
drwxrwxrwx 1 root   qbio       300 May 30 19:47 @eaDir
-rw-r--r-- 1 lsj975 qbio       195 May 30 19:07 files.txt
-rw-r--r-- 1 lsj975 qbio 242463028 May 30 19:10 grch_lin28b_clip.bam
-rw-r--r-- 1 lsj975 qbio    781912 May 30 17:57 GSE37111_clip.txt.gz
-rwxrwxrwx 1 lsj975 qbio 775474314 May 30 19:25 SRR458759.fasta.gz


2. human annotation file에서 miR-let7g 찾아보기

In [1]:
!zcat /vault/db/gencode/human/latest_release/gencode.v45.annotation.gtf.gz |grep MIRLET7G

chr3	ENSEMBL	gene	52268278	52268361	.	-	.	gene_id "ENSG00000199150.3"; gene_type "miRNA"; gene_name "MIRLET7G"; level 3; hgnc_id "HGNC:31485";
chr3	ENSEMBL	transcript	52268278	52268361	.	-	.	gene_id "ENSG00000199150.3"; transcript_id "ENST00000362280.3"; gene_type "miRNA"; gene_name "MIRLET7G"; transcript_type "miRNA"; transcript_name "MIRLET7G-201"; level 3; transcript_support_level "NA"; hgnc_id "HGNC:31485"; tag "basic"; tag "Ensembl_canonical";
chr3	ENSEMBL	exon	52268278	52268361	.	-	.	gene_id "ENSG00000199150.3"; transcript_id "ENST00000362280.3"; gene_type "miRNA"; gene_name "MIRLET7G"; transcript_type "miRNA"; transcript_name "MIRLET7G-201"; exon_number 1; exon_id "ENSE00001437043.3"; level 3; transcript_support_level "NA"; hgnc_id "HGNC:31485"; tag "basic"; tag "Ensembl_canonical";


** index file을 새로 만들어주고 해당 좌표 라인을 parsing하기

In [4]:
!samtools view -X grch_lin28b_clip.sorted.bam grch_lin28b_clip.sorted.bam.bai chr3:52268278-52268361 |wc -l

362


In [7]:
!samtools view -X grch_lin28b_clip.sorted.bam grch_lin28b_clip.sorted.bam.bai chr3:52268278-52268361 -o lin28b_mirlet7g_clip.bam